### Install requirements

First, run the cells below to install the requirements:

## Fine-tune large models using 🤗 `peft` adapters, `transformers` & `bitsandbytes`

In this tutorial we will cover how we can fine-tune large language models using the very recent `peft` library and `bitsandbytes` for loading large models in 8-bit.
The fine-tuning method will rely on a recent method called "Low Rank Adapters" (LoRA), instead of fine-tuning the entire model you just have to fine-tune these adapters and load them properly inside the model. 
After fine-tuning the model you can also share your adapters on the 🤗 Hub and load them very easily. Let's get started!

In [1]:
!pip install -q bitsandbytes datasets accelerate loralib
!pip install -q  git+https://github.com/zphang/peft.git@llama
!pip install -q git+https://github.com/zphang/transformers.git@llama_push


### Model loading

Here let's load the `opt-6.7b` model, its weights in half-precision (float16) are about 13GB on the Hub! If we load them in 8-bit we would require around 7GB of memory instead.

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import torch
import torch.nn as nn
import bitsandbytes as bnb
from transformers import LLaMAForCausalLM, LLaMATokenizer, DataCollatorForSeq2Seq,TrainingArguments, Trainer

torch.backends.cuda.matmul.allow_tf32 = True

model = LLaMAForCausalLM.from_pretrained(
    "decapoda-research/llama-7b-hf", 
    load_in_8bit=True, 
    device_map='auto',
)

tokenizer = LLaMATokenizer.from_pretrained("decapoda-research/llama-7b-hf")
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
model.resize_token_embeddings(len(tokenizer))


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching /usr/local/cuda/lib64...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.0
CUDA SETUP: Detected CUDA version 112
CUDA SETUP: Loading binary /usr/local/lib/python3.9/dist-packages/bitsandbytes/libbitsandbytes_cuda112.so...


/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/usr/local/nvidia/lib64'), PosixPath('/usr/local/nvidia/lib')}
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: /usr/local/nvidia/lib:/usr/local/nvidia/lib64 did not contain libcudart.so as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('443'), PosixPath('tcp'), PosixPath('//10.43.0.1')}
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('module'), PosixPath('//matplotlib_inline.backend_inline')}
  warn(msg)
Overriding tor

Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.


Embedding(32001, 4096)

### Post-processing on the model

Finally, we need to apply some post-processing on the 8-bit model to enable training, let's freeze all our layers, and cast the layer-norm in `float32` for stability. We also cast the output of the last layer in `float32` for the same reasons.

In [3]:
for param in model.parameters():
  param.requires_grad = False  # freeze the model - train adapters later
  if param.ndim == 1:
    # cast the small parameters (e.g. layernorm) to fp32 for stability
    param.data = param.data.to(torch.float32)

model.gradient_checkpointing_enable()  # reduce number of stored activations
model.enable_input_require_grads()

class CastOutputToFloat(nn.Sequential):
  def forward(self, x): return super().forward(x).to(torch.float32)
model.lm_head = CastOutputToFloat(model.lm_head)

### Apply LoRA

Here comes the magic with `peft`! Let's load a `PeftModel` and specify that we are going to use low-rank adapters (LoRA) using `get_peft_model` utility function from `peft`.

In [4]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [5]:
from peft import LoraConfig, get_peft_model 

config = LoraConfig(
    r=4,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 2097152 || all params: 6740520960 || trainable%: 0.03111261002591705


In [6]:
import bitsandbytes as bnb
from torch import nn
from transformers.trainer_pt_utils import get_parameter_names

training_args =TrainingArguments(
        per_device_train_batch_size=4, 
        gradient_accumulation_steps=4,
        warmup_steps=100, 
        max_steps=200, 
        learning_rate=1e-4, 
        tf32=True,
        logging_steps=1, 
        output_dir='outputs'
    )

decay_parameters = get_parameter_names(model, [nn.LayerNorm])
decay_parameters = [name for name in decay_parameters if "bias" not in name]
optimizer_grouped_parameters = [
    {
        "params": [p for n, p in model.named_parameters() if n in decay_parameters],
        "weight_decay": training_args.weight_decay,
    },
    {
        "params": [p for n, p in model.named_parameters() if n not in decay_parameters],
        "weight_decay": 0.0,
    },
]

optimizer_kwargs = {
    "betas": (training_args.adam_beta1, training_args.adam_beta2),
    "eps": training_args.adam_epsilon,
}
optimizer_kwargs["lr"] = training_args.learning_rate
adam_bnb_optim = bnb.optim.Adam8bit(
    optimizer_grouped_parameters,
    betas=(training_args.adam_beta1, training_args.adam_beta2),
    eps=training_args.adam_epsilon,
    lr=training_args.learning_rate,
)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /home/jovyan/.cache/huggingface/datasets/json/default-d3b399ca51002110/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [42]:
#todo dynamically split the dataset based on keys
def get_max_column_length(tokenizer, dataset,column_name):
    tokenized_inputs = (dataset).map(lambda x: tokenizer(x[column_name], truncation=True, padding='max_length'), batched=True)
    max_source_length = max([len(x['input_ids']) for x in tokenized_inputs])
    return max_source_length


def preprocess(sample,max_source_length,max_target_length,tokenizer,input_column_name,target_column_name,pading="max_length"):
    # add prefix for code2text task
    inputs = [item for item in sample[input_column_name]]
    #tokenize inputs
    model_inputs = tokenizer(inputs, max_length=max_source_length, padding=pading, truncation=True)
    #tokenize targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(sample[target_column_name], max_length=max_target_length, padding=pading, truncation=True)
    #set labels
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

def load_prompt_file(filepath):
        return open(filepath, 'r').read()




In [36]:
def generate_training_prompt(item,prompt_a,prompt_b):
    instruction =item['instruction']
    input = item['input']

    #if item object has input use prompt a else use prompt b
    if len(input)>0:
        return prompt_a.format(instruction,input)
    else:
        return prompt_b.format(instruction)
def generate_json(item,prompt_a,prompt_b):
    return {
        "text": generate_training_prompt(item,prompt_a,prompt_b),
        "target": item['output']
    }
def load_prompt_file(filepath):
        return open(filepath, 'r').read()

prompt_with_input = load_prompt_file('prompt_with_input.txt')
prompt_without_input = load_prompt_file('prompt_without_input.txt')



In [41]:
import json
from datasets import Dataset
alpaca_dataset_path = 'alpaca_data.json'

def load_json_file(file_path):
    with open(file_path, 'r') as file:
        return json.load(file)


json_objects = load_json_file(alpaca_dataset_path)
print(len(json_objects))
mapped_json_objects = [generate_json(item,prompt_with_input,prompt_without_input) for item in json_objects]
mapped_data = {'text': [], 'target': []}
for example in mapped_json_objects:
    mapped_data['text'].append(example['text'])
    mapped_data['target'].append(example['target'])

dataset = Dataset.from_dict(mapped_data)
print(dataset)

52002
Dataset({
    features: ['text', 'target'],
    num_rows: 52002
})


In [43]:
max_source_lenght = get_max_column_length(tokenizer,dataset,'text')
max_target_lenght = get_max_column_length(tokenizer,dataset,'target')
#preprocess dataset
tokenized_dataset = dataset.map(lambda x: preprocess(x,max_source_length,max_target_length,prefix,tokenizer,input_column_name,target_column_name),batched=True,remove_columns=train.column_names)
#adds padding to the labels
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, label_pad_token_id=label_pad_token_id)


NameError: name 'tokenizer' is not defined

### Training

In [ ]:



trainer = Trainer(
    model=model, 
    train_dataset=data['train'],
    optimizers=(adam_bnb_optim, None),
    args=TrainingArguments(
        gradient_checkpointing=True,
        per_device_train_batch_size=8, 
        gradient_accumulation_steps=8,
        warmup_steps=100, 
        max_steps=12500, 
        learning_rate=1e-4, 
        tf32=True,
        bf16=True,
        logging_steps=10, 
        output_dir='outputs'
    ),
    data_collator,
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

Using custom data configuration Abirate--english_quotes-7ef692ccb59fbf2a
Reusing dataset json (/root/.cache/huggingface/datasets/Abirate___json/Abirate--english_quotes-7ef692ccb59fbf2a/0.0.0/da492aad5680612e4028e7f6ddc04b1dfcec4b64db470ed7cc5f2bb265b9b6b5)


  0%|          | 0/1 [00:00<?, ?it/s]

Parameter 'function'=<function <lambda> at 0x7f022aad8040> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/3 [00:00<?, ?ba/s]

Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


988


  0%|          | 0/3 [00:00<?, ?ba/s]

wandb: Currently logged in as: epinnock (aetherfoundry). Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.9/dist-packages/bitsandbytes/autograd/_functions.py:298: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.9/dist-packages/bitsandbytes/autograd/_functions.py:298: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss
10,1.355700
20,1.412500
30,1.248500
40,1.208700
50,1.087900
60,1.012600
70,0.972900
80,1.064100


## Share adapters on the 🤗 Hub

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
model.push_to_hub("epinnock/llama7b-lora", use_auth_token=True)

Uploading the following files to ybelkada/opt-6.7b-lora: adapter_config.json,adapter_model.bin


Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

adapter_model.bin:   0%|          | 0.00/33.6M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/ybelkada/opt-6.7b-lora/commit/6f240b184e666b54a51b3fe482e4711448e6c751', commit_message='Upload model', commit_description='', oid='6f240b184e666b54a51b3fe482e4711448e6c751', pr_url=None, pr_revision=None, pr_num=None)

## Load adapters from the Hub

You can also directly load adapters from the Hub using the commands below:

In [ ]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

peft_model_id = "epinnock/llama7b-lora"
config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, return_dict=True, load_in_8bit=True, device_map='auto')
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

# Load the Lora model
model = PeftModel.from_pretrained(model, peft_model_id)


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
For effortless bug reporting copy-paste your error into this form: https://docs.google.com/forms/d/e/1FAIpQLScPB8emS3Thkp66nvqwmjTEgxp8Y9ufuWTzFyr9kJ5AoI47dQ/viewform?usp=sf_link
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching /usr/local/cuda/lib64...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 112
CUDA SETUP: Loading binary /usr/local/lib/python3.8/dist-packages/bitsandbytes/libbitsandbytes_cuda112.so...


/usr/local/lib/python3.8/dist-packages/bitsandbytes/cuda_setup/main.py:134: UserWarning: /usr/lib64-nvidia did not contain libcudart.so as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.8/dist-packages/bitsandbytes/cuda_setup/main.py:134: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.8/dist-packages/bitsandbytes/cuda_setup/main.py:134: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/usr/share/tcltk/tcllib1.19')}
  warn(msg)
/usr/local/lib/python3.8/dist-packages/bitsandbytes/cuda_setup/main.py:134: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('--listen_host=172.28.0.12 --target_host=172.28.0.12 --tunnel_background_save_url=https'), PosixPath('//colab.resear

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

## Inference

You can then directly use the trained model or the model that you have loaded from the 🤗 Hub for inference as you would do it usually in `transformers`.

In [9]:
batch = tokenizer("If you tell the truth, you", return_tensors='pt').to('cuda')
with torch.no_grad():
    with torch.cuda.amp.autocast():
        output_tokens = model.generate(**batch, max_new_tokens=50, pad_token_id=tokenizer.eos_token_id,  
            no_repeat_ngram_size=2,       
            do_sample=True, 
            top_k=3, 
            top_p=0.7,
            temperature=0.8)

print('\n\n', tokenizer.decode(output_tokens[0], skip_special_tokens=True))

/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1201: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(




  If you tell the truth, you don't have to remember anything. - Mark Twain
If you're going to tell people the Truth, be ready to suffer and pay the price. It's not enough to just tell it: You've got to be


As you can see by fine-tuning for few steps we have almost recovered the quote from Albert Einstein that is present in the [training data](https://huggingface.co/datasets/Abirate/english_quotes).